![A soccer pitch for an international match.](soccer-pitch.jpg)

You're working as a sports journalist at a major online sports media company, specializing in soccer analysis and reporting. You've been watching both men's and women's international soccer matches for a number of years, and your gut instinct tells you that more goals are scored in women's international football matches than men's. This would make an interesting investigative article that your subscribers are bound to love, but you'll need to perform a valid statistical hypothesis test to be sure!

While scoping this project, you acknowledge that the sport has changed a lot over the years, and performances likely vary a lot depending on the tournament, so you decide to limit the data used in the analysis to only official `FIFA World Cup` matches (not including qualifiers) since `2002-01-01`.

You create two datasets containing the results of every official men's and women's international football match since the 19th century, which you scraped from a reliable online source. This data is stored in two CSV files: `women_results.csv` and `men_results.csv`.

The question you are trying to determine the answer to is:

> Are more goals scored in women's international soccer matches than men's?

You assume a **10% significance level**, and use the following null and alternative hypotheses:

$H_0$ : The mean number of goals scored in women's international soccer matches is the same as men's.

$H_A$ : The mean number of goals scored in women's international soccer matches is greater than men's.

In [55]:
# Load all necessary packages
import numpy as np
import pandas as pd
import csv
import pingouin
from scipy.stats import mannwhitneyu

In [56]:
# Define a function to remove the first column from a CSV file and return a DataFrame
def remove_first_column(file_path):
    with open(file_path, 'r') as file:
        reader = csv.reader(file)
        header = next(reader)[1:]  # Read the header
        data = [row[1:] for row in reader]  # Read the remaining rows, skipping the first column
    return pd.DataFrame(data, columns=header)

# Apply the function to CSV
men_df = remove_first_column('men_results.csv')
women_df = remove_first_column('women_results.csv')
men_df.info() # Check data

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44353 entries, 0 to 44352
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   date        44353 non-null  object
 1   home_team   44353 non-null  object
 2   away_team   44353 non-null  object
 3   home_score  44353 non-null  object
 4   away_score  44353 non-null  object
 5   tournament  44353 non-null  object
dtypes: object(6)
memory usage: 2.0+ MB


In [57]:
# Typecast necessary columns from both datasets
men_df['date'] = pd.to_datetime(men_df['date'])
men_df[['home_score', 'away_score']] = men_df[['home_score', 'away_score']].astype('int')
women_df['date'] = pd.to_datetime(women_df['date'])
women_df[['home_score', 'away_score']] = women_df[['home_score', 'away_score']].astype('int')

# Recheck changes
men_df.info() # Recheck changes
women_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44353 entries, 0 to 44352
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   date        44353 non-null  datetime64[ns]
 1   home_team   44353 non-null  object        
 2   away_team   44353 non-null  object        
 3   home_score  44353 non-null  int64         
 4   away_score  44353 non-null  int64         
 5   tournament  44353 non-null  object        
dtypes: datetime64[ns](1), int64(2), object(3)
memory usage: 2.0+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4884 entries, 0 to 4883
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   date        4884 non-null   datetime64[ns]
 1   home_team   4884 non-null   object        
 2   away_team   4884 non-null   object        
 3   home_score  4884 non-null   int64         
 4   away_score  4884 non-null   int64       

In [58]:
# Subset the date and tournament specifics for the analysis
filtered_men_df = men_df[(men_df['date'] > "2002-01-10") & (men_df['tournament'] == 'FIFA World Cup')]
filtered_women_df = women_df[(women_df['date'] > "2002-01-10") & (women_df['tournament'] == 'FIFA World Cup')]

# Create identifier columns for men and women goal scores
filtered_men_df['gender'] = 'men'
filtered_women_df['gender'] = 'women'

# Add the total goal scores of the teams (home and away scores)
filtered_men_df['total_goals'] = filtered_men_df['home_score'] + filtered_men_df['away_score']
filtered_women_df['total_goals'] = filtered_women_df['home_score'] + filtered_women_df['away_score']

# Display the filtered datasets
print(filtered_men_df.head())
print(filtered_women_df.head())

            date            home_team  ... gender  total_goals
25164 2002-05-31               France  ...    men            1
25165 2002-06-01              Germany  ...    men            8
25166 2002-06-01  Republic of Ireland  ...    men            2
25167 2002-06-01              Uruguay  ...    men            3
25168 2002-06-02            Argentina  ...    men            1

[5 rows x 8 columns]
           date      home_team    away_team  ...      tournament  gender total_goals
1600 2003-09-20        Nigeria  North Korea  ...  FIFA World Cup   women           3
1601 2003-09-20         Norway       France  ...  FIFA World Cup   women           2
1602 2003-09-20        Germany       Canada  ...  FIFA World Cup   women           5
1603 2003-09-20          Japan    Argentina  ...  FIFA World Cup   women           6
1604 2003-09-21  United States       Sweden  ...  FIFA World Cup   women           4

[5 rows x 8 columns]


In [59]:
# Concatenate the two tables for hypothesis testing
combined_df = pd.concat([filtered_men_df, filtered_women_df], axis= 0)
combined_subset = combined_df[['gender', 'total_goals']]
combined_subset_wide = combined_subset.pivot(columns= 'gender', values= 'total_goals')
combined_subset_wide

gender,men,women
1600,NaN,3.0
1601,NaN,2.0
1602,NaN,5.0
1603,NaN,6.0
1604,NaN,4.0
...,...,...
44343,3.0,NaN
44345,3.0,NaN
44346,2.0,NaN
44350,3.0,NaN


In [60]:
# Conduct Wilcoxon-Mann-Whitney test with pingouin since data is not normally distributed
test_results = pingouin.mwu(x= combined_subset_wide["women"],
                          y= combined_subset_wide["men"],
                          alternative= "greater")
alpha = 0.1 # Set alpha
p_value = test_results['p-val'].values[0] # Subset the p value

# Check for the hypothesis 
if p_value <= alpha:
    result = "reject"
else:
    result = "fail to reject"

# Create dictionary with Hypothesis result
result_dict = {
    "p_val": p_value,
    "result": result
}

print(result_dict)

{'p_val': 0.005106609825443641, 'result': 'reject'}
